In [ ]:
import numpy as np
import sys
import os
import time
import cv2
from google.colab import drive

# Mounting drive for extracting the files

In [ ]:
# Requires authentication on personal drive

drive.mount('/content/drive')
dbpath = '/content/drive/My Drive/CCSN'

# Defining relevant functions

In [ ]:
def Descriptori_LBP_RGB(matrix,t):
    # PURPOSE: computes the base LTP texture descriptor of a given image
    #
    # PARAMETERS: matrix: image to be extracted
    #            t: the selected noise threshold
    #
    # RETURNS: histcorob: the base LTP texture descriptor of the given image

    histcorob=np.zeros(1536,dtype=np.float64)
    t=np.float64(t)
    h,w = matrix[:,:,0].shape
    for plan_culoare in range(3):
      for i in range (1,h-1):
          for j in range(1,w-1):
              CODE_POZ=0
              CODE_NEG=0
              c=np.float64(matrix[i][j][plan_culoare])
      # "diffs" stores all the substractions made between the reference pixel and its "neighbours"
              diffs=np.float64([matrix[i][j-1][plan_culoare],
                                matrix[i+1][j-1][plan_culoare],
                                matrix[i+1][j][plan_culoare],
                                matrix[i+1][j+1][plan_culoare],
                                matrix[i][j+1][plan_culoare],
                                matrix[i-1][j+1][plan_culoare],
                                matrix[i-1][j][plan_culoare],
                                matrix[i-1][j-1][plan_culoare]])-np.float64(c)

              CODE_POZ+= 1<<7 if diffs[0]>=t else 0   #MSB
              CODE_NEG+= 1<<7 if diffs[0]<=-t else 0

              CODE_POZ+= 1<<6 if diffs[1]>=t else 0
              CODE_NEG+= 1<<6 if diffs[1]<=-t else 0

              CODE_POZ+= 1<<5 if diffs[2]>=t else 0
              CODE_NEG+= 1<<5 if diffs[2]<=-t else 0

              CODE_POZ+= 1<<4 if diffs[3]>=t else 0
              CODE_NEG+= 1<<4 if diffs[3]<=-t else 0

              CODE_POZ+= 1<<3 if diffs[4]>=t else 0
              CODE_NEG+= 1<<3 if diffs[4]<=-t else 0

              CODE_POZ+= 1<<2 if diffs[5]>=t else 0
              CODE_NEG+= 1<<2 if diffs[5]<=-t else 0

              CODE_POZ+= 1<<1 if diffs[6]>=t else 0
              CODE_NEG+= 1<<1 if diffs[6]<=-t else 0

              CODE_POZ+= 1 if diffs[7]>=t else 0
              CODE_NEG+= 1 if diffs[7]<=-t else 0     #LSB

              histcorob[CODE_POZ + 256 * (plan_culoare * 2) ]+=1
              histcorob[CODE_NEG + 256 * (plan_culoare * 2 + 1) ]+=1

    return histcorob

# The File Extraction Procedure

In [ ]:
# variable keeping track of extracted files in one session (useful in case of a keyboard interrupt)
# if session times out and variable workspace is cleared, file extraction progression needs to be checked
# manually in the mounted drive in order to resume proper extraction

IS_EXTRACTED = {'Ac':False,'As':False,'Cb':False,'Cc':False,'Ci':False,'Cs':False,'Ct':False,'Cu':False,'Ns':False,'Sc':False,'St':False}

In [ ]:
# If you want to test this file extraction procedure for yourself,
# set up a dummy folder with the name 'CCSN' with a few mock images
# for extraction in the 'My Drive' folder.
#
# Images are extracted in .binary and stored in format np.float64

ZGOMOT = 2                                       # noise threshold (NEEDS TO BE SET MANUALLY)
DBPATH = '/content/drive/My Drive/CCSN'          # image database source
BACKPATH = '/content/drive/My Drive/backups'     # extracted file destination
BACKFOLDER = 'ZGOMOT_' + str(ZGOMOT)


contor = 0
print (f'EXTRACTING TEXTURE DESCRIPTORS FOR NOISE THRESHOLD {ZGOMOT} ')
for classdirectory in os.listdir(DBPATH):
    print(f'EXTRACTING CLASS {classdirectory}...')
    BACKFILE =  'LTP_' + str(ZGOMOT) + '_' + classdirectory + '.bin'
    if IS_EXTRACTED[classdirectory]:
      print(f'Class {classdirectory} has already been extracted. Continuing...\n')
      continue
    else:
      with open (os.path.join(BACKPATH,BACKFOLDER,BACKFILE),'a') as file:
        contor = 0
        for image in os.listdir(os.path.join(DBPATH,classdirectory)):
          contor+=1

          imagine_extrasa = cv2.cvtColor(cv2.imread(os.path.join(DBPATH,classdirectory,image)),cv2.COLOR_BGR2RGB)
          Descriptori_LBP_RGB (imagine_extrasa , ZGOMOT).tofile(file) #histograma extrasa scrisa in fisier

          if contor%10 == 0 : print(f'{contor} {classdirectory} extracted images...')

      print(f'Class {classdirectory} has been extracted!\n')
      IS_EXTRACTED[classdirectory] = True
print('END OF EXTRACTION')

